In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
import matplotlib.pyplot as plt
import winsound

In [2]:
def live_mask_detector(configure_file, model_file, model):
    '''
    In this function we are Loading our prototxt and caffe model. 
    Then we use camera and process frames, extract the dimensions, resize to 300px and create a blob using DNN.
    Made prediction using our classifier and displayed output as text over video output.
    '''    
    
    cap = cv2.VideoCapture(0) #reading video from system camera
    net=cv2.dnn.readNetFromCaffe(configure_file,model_file) #loading model
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480)) #to save output
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read() #ret is True if read correctly

        h,w=frame.shape[:2]

        #preprocessing frame

        blob=cv2.dnn.blobFromImage(frame,scalefactor = 1.0, size = (300,300), mean = (104.0, 177.0, 123.0))

        #passing blob through the network to detect and prediction
        net.setInput(blob)
        detections=net.forward()
        
        #using a try block to ensure if any error occurs then the camera must be released
        try:

            #we’ll loop over the detections and draw boxes around the detected faces with confidence >0.5
            for i in range(detections.shape[2]):

                confidence=detections[0,0,i,2]
                if confidence>0.25:
                    #detection height and width w.r.t frame is alwayas within 0 and 1
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")


                    #making sure starting and ending pixels lie inside image
                    (startX, startY) = (max(0, startX), max(0, startY))
                    (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

                    #extracting face
                    face = frame[startY:endY, startX:endX]

                    #preprocessing face for classifer model
                    face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB) #switching channels
                    face=cv2.resize(face,(150,150))
                    face = preprocess_input(face)
                    face = np.expand_dims(face, axis=0)
                    pred = model.predict(face) #predicting mask or no mask by passing extracted face from image

                    proba=1-pred #as we have class 0 for maska and 1 for no mask

                    label = "Mask" if pred < 0.4 else "No Mask"

                    # plays a beep sound when there is no mask
#                     if label == "No Mask":
#                         frequency = 2500  # Set Frequency To 2500 Hertz
#                         duration = 1000  # Set Duration To 1000 ms == 1 second
#                         winsound.Beep(frequency, duration) #this will work in windows only.

                    color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
                    label=label+' '+str((proba[0][0]*100).round(3))+'%'

                    cv2.putText(frame, label, (startX, startY-10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)
                    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
                
        except :
            break


        # Display the resulting frame
        out.write(frame)
        cv2.imshow("prediction", frame)



        if cv2.waitKey(1) & 0xFF == ord('q'): #pressing q will terminate the process
            break

    # When everything done, release the capture
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [3]:
configure_file=r'face_detector\deploy.prototxt' #this file contain model architecture
model_file=r'face_detector\res10_300x300_ssd_iter_140000.caffemodel' #this file conatin model weight

In [4]:
model=tf.keras.models.load_model('classifier.h5')

In [7]:
live_mask_detector(configure_file, model_file, model)